In [33]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from datetime import datetime


#Leggo il file in output di chatGPT
chatGPTOutpt = pd.read_csv('../Query/output/outputChatGPT.csv')

#Leggo le annotazioni parsate degli annotatori
annotazioni = pd.read_csv('../Annotations/Annotations_checked.csv')

In [40]:
chatGPTOutpt

,id,title,label,annotator,Q1.1,Q1.2,Q1.3.1,Q1.4,Q1.5,Q1.6,...,Q1.7.1_3,Q1.7.1_4,Q1.7.1_5,Q1.7.1_6,Q1.7.1_7,Q1.7.1_8,Q1.7.1_9,Q1.7.1_10,Q1.7.1_11,Q1.7.1_12
0,3d1ae696-3a3f-43ab-a4ce-61d337a59d7e,"Strage nel Mediterraneo: barcone si ribalta, m...",Group 2,ChatGPT4AsAnnotator,1,4,2,2,4,3,...,0,1,0,0,0,0,0,0,0,0
1,7b64542e-11d0-464d-a060-1145a9d0ad3a,"Il premier libico a Roma, dalla ricostruzione ...",Group 2,ChatGPT4AsAnnotator,3,4,2,2,4,3,...,0,0,0,0,0,0,0,0,0,0
2,d73018e2-5184-43b7-ba01-e82378666dab,In Fvg trovata la variante Delta in 25 dei 96 ...,Group 2,ChatGPT4AsAnnotator,1,4,2,2,4,3,...,0,0,0,0,0,0,0,0,0,0
3,1d134854-568a-4590-86b1-64912b3c3e4d,Nuove aggressioni omofobe a Milano: calci e pu...,Group 2,ChatGPT4AsAnnotator,3,4,2,2,3,3,...,1,0,0,0,0,1,0,0,0,0


In [34]:
# Conformo l'output di chatGPT a quello degli annotatori per fare il confronto


# Funzione per convertire i valori
def convert_values(value):
    if isinstance(value, str):
        # Rimuovi parentesi quadre
        value = value.strip("[]")
        # Sostituisci virgole con punti e virgola
        value = value.replace(",", ";")
        value = value.replace("'", "")
        value = value.replace(" ", "")

    return value

# Applica la funzione alla colonna
chatGPTOutpt['Q1.7.1'] = chatGPTOutpt['Q1.7.1'].apply(convert_values)
chatGPTOutpt['Q1.3'] = chatGPTOutpt['Q1.3'].apply(convert_values)



####### CASTO I TIPI E PULISCO ########


####
# Converto le colonne numeriche in INT
####
columns_to_convert = ['Q1.1', 'Q1.2', 'Q1.3.1', 'Q1.4', 'Q1.5', 'Q1.6', 'Q1.7', 'Q2.8', 'Q2.9.1', 'Q2.10']
# Sotituisco i NaN in -1
chatGPTOutpt[columns_to_convert] = chatGPTOutpt[columns_to_convert].fillna(-1)
# Applico il cast
for col in columns_to_convert:
    chatGPTOutpt[columns_to_convert] = chatGPTOutpt[columns_to_convert].astype(int)


####################################################################################
# Faccio one hot delle colonne a valore multuiplo (Q1.3, 1.7.1)
####################################################################################
    
# Creiamo una nuova colonna per ciascun valore unico trovato nella colonna "Q1.3"
listQ_1_3 = [1,2,3,4,5,6,7,8,9]
for value in listQ_1_3:
    chatGPTOutpt[f'Q1.3_{value}'] = 0


# Creiamo una nuova colonna per ciascun valore unico trovato nella colonna "Q1.7"
listQ1_7_1 = [0,1,2,3,4,5,6,7,8,9,10,11,12]
for value in listQ1_7_1:
    chatGPTOutpt[f'Q1.7.1_{value}'] = 0

# Assegniamo 1 a ciascuna colonna appena creata se il valore corrisponde a quello nella colonna "Q1.3"
for idx, row in chatGPTOutpt.iterrows():
    values = str(row['Q1.3']).split(';')  # Convertiamo in stringa prima di fare lo split
    for value in values:
        chatGPTOutpt.at[idx, f'Q1.3_{value}'] = 1


# Assegniamo 1 a ciascuna colonna appena creata se il valore corrisponde a quello nella colonna "Q1.7.1"
for idx, row in chatGPTOutpt.iterrows():
    values = str(row['Q1.7.1']).split(';')  # Convertiamo in stringa prima di fare lo split
    for value in values:
        chatGPTOutpt.at[idx, f'Q1.7.1_{value}'] = 1


#elimino le colonne ridondanti
chatGPTOutpt.drop(['Q1.3','Q1.7.1', 'Q1.7.1_-1', 'Problem'], axis=1, inplace=True)




In [36]:
# Faccio il merge dei due csv

# Esegui il merge specificando le colonne comuni come chiavi di join e usando un dizionario per specificare le colonne da unire
frames = [annotazioni, chatGPTOutpt]

df = pd.concat(frames)

In [39]:
df.to_csv('tosee.csv')

In [46]:
# Per ciascun articolo condiviso, calcola il coefficiente Kappa di Cohen
def getKappa (annotator_1, annotator_2):
# Inizializza una lista per memorizzare i coefficienti Kappa di Cohen
    kappa_scores = []

    articles = set(df[df['annotator'] == annotator_1]['id']).intersection(
    set(df[df['annotator'] == annotator_2]['id']))
    for article_id in articles:
        annotator1 = df[(df['id'] == article_id) & (df['annotator'] == annotator_1)][['Q1.1', 'Q1.2', 'Q1.3.1', 'Q1.4', 'Q1.5', 'Q1.6', 'Q1.7', 'Q2.8', 'Q2.9.1', 'Q2.9.2', 'Q2.10','Q1.3_1','Q1.3_2','Q1.3_3','Q1.3_4','Q1.3_5','Q1.3_6','Q1.3_7','Q1.3_8','Q1.3_9','Q1.7.1_0','Q1.7.1_1','Q1.7.1_2','Q1.7.1_3','Q1.7.1_4','Q1.7.1_5','Q1.7.1_6','Q1.7.1_7','Q1.7.1_8','Q1.7.1_9','Q1.7.1_10','Q1.7.1_11','Q1.7.1_12']]
        annotator2 = df[(df['id'] == article_id) & (df['annotator'] == annotator_2)][['Q1.1', 'Q1.2', 'Q1.3.1', 'Q1.4', 'Q1.5', 'Q1.6', 'Q1.7', 'Q2.8', 'Q2.9.1', 'Q2.9.2', 'Q2.10','Q1.3_1','Q1.3_2','Q1.3_3','Q1.3_4','Q1.3_5','Q1.3_6','Q1.3_7','Q1.3_8','Q1.3_9','Q1.7.1_0','Q1.7.1_1','Q1.7.1_2','Q1.7.1_3','Q1.7.1_4','Q1.7.1_5','Q1.7.1_6','Q1.7.1_7','Q1.7.1_8','Q1.7.1_9','Q1.7.1_10','Q1.7.1_11','Q1.7.1_12']]

        if ((annotator1['Q2.9.2'] != -1) & (annotator2['Q2.9.2'] != -1)).all():
            annotator1['Q2.9.2'] = pd.to_datetime(annotator1['Q2.9.2'])
            annotator2['Q2.9.2'] = pd.to_datetime(annotator2['Q2.9.2'])
            annotator1['Q2.9.2'] = annotator1['Q2.9.2'].astype('int64')
            annotator2['Q2.9.2'] = annotator2['Q2.9.2'].astype('int64')

        else:
            annotator1.drop(['Q2.9.2'], axis=1, inplace=True)
            annotator2.drop(['Q2.9.2'], axis=1, inplace=True)

        # Converti i valori in numeri interi o float
        annotator1 = annotator1.astype(float)
        annotator2 = annotator2.astype(float)
        
        # Calcola il coefficiente Kappa di Cohen per ciascuna coppia di annotatori
        k = cohen_kappa_score(annotator1.values.flatten(), annotator2.values.flatten())
        kappa_scores.append(k)
    if (len(kappa_scores) > 0):
        mean_kappa = sum(kappa_scores) / len(kappa_scores)
    else:
        print('Non ci sono annotazioni in comune tra gli annotatori ' + str(annotator_1) + ' e ' + str(annotator_1))
        return ()
    print ('Il K tra ' + str(annotator_1) +' e ' + str(annotator_2) +' è  di: '  + str(mean_kappa))
    return mean_kappa

k_manuel_angelo = getKappa('Manuel', 'Angelo')
k_manuel_marinella = getKappa('Manuel', 'Marinella')
k_angelo_marinella = getKappa('Angelo', 'Marinella')
k_gpt_marinella = getKappa('ChatGPT4AsAnnotator', 'Marinella')
k_gpt_angelo = getKappa('ChatGPT4AsAnnotator', 'Angelo')
k_gpt_manuel = getKappa('ChatGPT4AsAnnotator', 'Manuel')



Il K tra Manuel e Angelo è  di: 0.8619795552450444
Il K tra Manuel e Marinella è  di: 0.8723454045098786
Il K tra Angelo e Marinella è  di: 0.8635265076197413
Il K tra ChatGPT4AsAnnotator e Marinella è  di: 0.5504288671990535
Il K tra ChatGPT4AsAnnotator e Angelo è  di: 0.7287000905084796
Il K tra ChatGPT4AsAnnotator e Manuel è  di: 0.6624934178808322
